<a href="https://colab.research.google.com/github/veritaem/DS-Unit-2-Sprint-4-Model-Validation/blob/master/Unit_2_Sprint_4_Mod_1_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#imports and a couple global models

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [0]:
lin_mod = LinearRegression()
log_mod = LogisticRegression()

In [17]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/treselle-systems/'
    'customer_churn_analysis/master/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


#pipeline attempt

In [7]:
!pip install category_encoders

In [0]:
import category_encoders as ce

In [0]:
pipeline = make_pipeline(
                          ce.OneHotEncoder(use_cat_names = True),
                          StandardScaler(),
                          LogisticRegression())

#crude baselines

In [0]:
df.Churn.replace({'Yes':1, 'No':0}, inplace = True)

In [19]:
df.Churn.mean()#so we see that most ppl(73% and some change) do not churn

0.2653698707936959

In [20]:
from sklearn.metrics import mean_absolute_error
baseline = [df.Churn.mean()] * len(df.Churn)  
mean_absolute_error(df.Churn, baseline)

0.38989740493726605

In [0]:
X = df.drop(['Churn', 'TotalCharges'], axis = 1)
y = df['Churn']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [23]:
maj_classification = y_train.mode()[0]
y_pred = np.full(shape=y_test.shape, fill_value=maj_classification)

accuracy_score(y_test, y_pred)

0.7399204997160704

In [24]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.8012492901760363

so my model will have to beat:

a MSE of 0.39, 

an accuracy of 74%, and

 a simple pipelined accuracy of 80%

#train_val_test split